In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# STEP 1 [install req. libraries]

In [ ]:
pip install numpy scipy matplotlib

# STEP 2 [ basic implementation of reaction-diffusion model ]

Please note that this simplified version mainly focuses on HGG growth. I intend to build off this model once I acuquire the necessary pHGG data, possible addition of patient specifc parameters / dependencies to aid in the models' clinical utlilty.

In [ ]:
import numpy as np
import scipy.ndimage as nd
import matplotlib.pyplot as plt


size = 256
U = np.ones((size, size)) * 0.5  # Initial tumor cell concentration
V = np.zeros((size, size))  # Initial environmental factor

# Define diffusion and reaction parameters
Du = 0.1  # Diffusion coefficient for tumor cells
Dv = 0.05  # Diffusion coefficient for environment factor
F = 0.035  # Feed rate (proliferation rate)
k = 0.065  # Kill rate (cell death rate)
dt = 1.0  # Time step

# Define the reaction-diffusion step function
def reaction_diffusion_step(U, V, Du, Dv, F, k, dt):
    laplacian_U = nd.laplace(U)
    laplacian_V = nd.laplace(V)
    
    dU = (Du * laplacian_U - U * V**2 + F * (1 - U)) * dt
    dV = (Dv * laplacian_V + U * V**2 - (F + k) * V) * dt
    
    U += dU
    V += dV
    return U, V

In [ ]:
num_steps = 10000  # Number of time steps
for step in range(num_steps):
    U, V = reaction_diffusion_step(U, V, Du, Dv, F, k, dt)
    if step % 1000 == 0:
        plt.imshow(U, cmap='hot', interpolation='nearest')
        plt.title(f"Step {step}")
        plt.show()

# Final visualization
plt.imshow(U, cmap='hot', interpolation='nearest')
plt.title("Final Tumor Cell Concentration")
plt.colorbar()
plt.show()

In [ ]:
pip install opencv-python

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Function to load and preprocess the image
def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load image in grayscale
    image = cv2.resize(image, (256, 256))  # Resize to match model grid size
    image = image / 255.0  # Normalize pixel values to [0, 1]
    return image

# Load your initial tumor data
initial_tumor_image_path = 'kaggle/input/mri-image-data/Testing/glioma_tumor/image(15).jpg'
initial_tumor_concentration = load_and_preprocess_image'kaggle/input/mri-image-data/Testing/glioma_tumor/image(15).jpg'

# Display the initial tumor concentration
plt.imshow(initial_tumor_concentration, cmap='hot', interpolation='nearest')
plt.title("Initial Tumor Cell Concentration")
plt.colorbar()
plt.show()

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import scipy.ndimage as nd

# Function to load and preprocess JPG images
def load_and_preprocess_jpg_image(file_path):
    image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)  # Load image in grayscale
    if image is None:
        raise ValueError(f"Image not found or cannot be read at path: {file_path}")
    image = cv2.resize(image, (256, 256))  # Resize to match model grid size
    image = image / 255.0  # Normalize pixel values to [0, 1]
    return image

# Example path to a JPG file
jpg_file_path = '/kaggle/input/mri-image-data/Testing/glioma_tumor/image(15).jpg'
try:
    tumor_image = load_and_preprocess_jpg_image(jpg_file_path)
    plt.imshow(tumor_image, cmap='hot', interpolation='nearest')
    plt.title("Preprocessed Tumor Image")
    plt.colorbar()
    plt.show()
except ValueError as e:
    print(e)

# Initialize the grid with real data
U = tumor_image  # Initial tumor cell concentration from real data
V = np.zeros((256, 256))  # Initial environmental factor

# Define diffusion and reaction parameters
Du = 0.1  # Diffusion coefficient for tumor cells
Dv = 0.05  # Diffusion coefficient for environment factor
F = 0.035  # Feed rate (proliferation rate)
k = 0.065  # Kill rate (cell death rate)
dt = 1.0  # Time step

# Define the reaction-diffusion step function
def reaction_diffusion_step(U, V, Du, Dv, F, k, dt):
    laplacian_U = nd.laplace(U)
    laplacian_V = nd.laplace(V)
    
    dU = (Du * laplacian_U - U * V**2 + F * (1 - U)) * dt
    dV = (Dv * laplacian_V + U * V**2 - (F + k) * V) * dt
    
    U += dU
    V += dV
    return U, V

# Simulate the tumor growth
num_steps = 10000  # Number of time steps
for step in range(num_steps):
    U, V = reaction_diffusion_step(U, V, Du, Dv, F, k, dt)
    if step % 1000 == 0:
        plt.imshow(U, cmap='hot', interpolation='nearest')
        plt.title(f"Step {step}")
        plt.colorbar()
        plt.show()

# Final visualization
plt.imshow(U, cmap='hot', interpolation='nearest')
plt.title("Final Tumor Cell Concentration")
plt.colorbar()
plt.show()